# Create an interface between two materials with minimal strain

Use Zur and McGill superlattices matching [algorithm](https://doi.org/10.1063/1.3330840) to create interfaces between two materials using the Pymatgen [implementation](https://pymatgen.org/pymatgen.analysis.interfaces.html#pymatgen.analysis.interfaces.zsl).

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials
2. Create slabs for substrate and film using slab generator with specified parameters, view them and select terminations



## 1. Prepare the Environment

### 1.1. Install the Required Packages
Note: the step executes only when using Pyodide. For other environments (e.g. Colab, JupyterLab), the packages should be installed via `pip install .` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("create_interface_with_min_strain_zsl.ipynb", "../../config.yml")

### 1.2. Get input materials and assign `substrate` and `film`


In [ ]:
from mat3ra.made.material import Material
from utils.jupyterlite import get_data

# Get the list of input materials and load them into `materials_in` variable
get_data("materials_in", globals())
materials = list(map(Material, globals()["materials_in"]))
substrate = materials[0]
film = materials[1]

### 1.3. Visualize Substrate and Film

In [ ]:
from utils.visualize import visualize_materials as visualize
visualize([substrate, film], repetitions=[3, 3, 1], rotation="0x")

## 2. Create Slabs for the Interface

### 2.1. Create Substrate and Layer Slabs via SlabGenerator

In [ ]:
substrate_slab_generator = SlabGenerator(
    bulk=substrate,
    # the Miller indices of the interfacial plane of the substrate
    miller_indices=(1, 1, 1),
    # substrate thickness in layers
    thickness=3,
    # the matrix of the surface cell along the basal plane dimension
    xy_supercell_matrix=[[1,0], [0,1]],
)

film_slab_generator = SlabGenerator(
    bulk=film,
    # the Miller indices of the interfacial plane of the layer
    miller_indices=(0, 0, 1),
    # layer thickness in layers
    thickness=1,
    # the matrix of the surface cell along the basal plane dimension
    xy_supercell_matrix=[[1,0], [0,1]],
)

substrate_slab_terminations = substrate_slab_generator.get_terminations()
film_slab_terminations = film_slab_generator.get_terminations()

### 2.2. Visualize slabs for possible terminations

In [ ]:
for termination in substrate_slab_terminations:
    visualize(substrate_slab_generator.get_slab(termination=termination), repetitions=[3, 3, 1], rotation="-90x", title=f"Termination: {termination}")
    
for termination in film_slab_terminations:
    visualize(film_slab_generator.get_slab(termination=termination), repetitions=[3, 3, 1], rotation="-90x", title=f"Termination: {termination}")
    

### 2.3. Create slabs with specified termination
Select termination that will be facing interfacial plane for substrate and film

In [ ]:
substrate_slab_termination = substrate_slab_terminations[0]
film_slab_termination = film_slab_terminations[0]

# Create substrate slab with the selected termination
substrate_slab = substrate_slab_generator.get_slab(termination=substrate_slab_terminations[0])
# Create film slab with the selected termination
film_slab_generator = film_slab_generator.get_slab(termination=film_slab_terminations[0])

## 3. Create an interface
### 3.1. Initialize the Interface Builder
We use no strain matching for the interface construction. Substrate slab cell is used as a base and the film slab is wrapped into it with optionally scaling atomic coordinates (preserve coordinates in crystal/alat units).

In [ ]:
from mat3ra.made.tools.build.interface import InterfaceBuilder, NoStrainMatchingParameters

no_strain_matching_parameters = NoStrainMatchingParameters.copy()
no_strain_matching_parameters.use_scaling_for_film = False

builder_with_no_strain_matching = InterfaceBuilder(
    substrate_slab=substrate_slab,
    film_parameters=film_slab_generator,
    # distance between substrate and layer, in Angstroms
    distance_z=3.0,
    # shift in x direction, in Angstroms
    shift_x=0.0,
    # shift in y direction, in Angstroms
    shift_y=0.0,
    # if True, the surface plane is constructed using miller indices of the conventional cell
    use_conventional_cell=True,
    use_strain_matching=False,
    strain_matching_parameters=no_strain_matching_parameters,
)

# 3.2. Create interface for the termination
Function `create_interfaces()` returns InterfaceDataHolder with list of interfaces for each termination when the strain matching algorithm is used. With no strain matching only one list item is returned.

In [ ]:
from mat3ra.made.tools.build import create_interfaces

interface_termination = (substrate_slab_termination, film_slab_termination)

interface = create_interfaces(
    interface_builder=builder_with_no_strain_matching,
    termination=interface_termination,
)[0]

### 6.2. Visualize the interface


In [ ]:
visualize(interface, repetitions=[1, 1, 1], rotation="0x")

### 6.3. Pass data to the outside runtime
Enrich the interface names with the strain values and pass them to the application runtime.

In [ ]:
from utils.jupyterlite import set_data

if "Interface, Strain:" not in interface["name"]:
    interface["name"] = f'{interface["name"]}, Interface, Strain:{interface["metadata"]["interface_properties"]["mean_abs_strain"]*100:.3f}%'

set_data("materials", [interface])